In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.0.1.post2
Torchvision Version:  0.2.1


In [2]:
from PIL import Image 
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
%matplotlib inline
trainLabels = pd.read_csv("./trainLabels.csv")

In [3]:
import copy

In [4]:
train_path = './train/train_224/*'
images_list = glob.glob(train_path)
images_list = sorted(images_list)

In [5]:
class EyeDataset(torch.utils.data.Dataset):
    """Face Landmarks dataset."""

    def __init__(self, images_list,labels,transform=None):
#         self.X = X
#         self.y = y
#         self.transform = transforms.Resize(224)
        self.images_list = images_list
        self.trainLables = labels
        self.transform = transform

    def __len__(self):
        print(len(self.images_list))
        return len(self.images_list)

    def __getitem__(self, idx):
#         print("index ",idx)
        imgtemp = Image.open(self.images_list[idx])
        if self.transform!=None :
            imgtemp = self.transform(imgtemp)
        img = transforms.ToTensor()(imgtemp)
#         fileName = self.images_list[idx].split('/')[-1].split('.')[0]
#         imglabel = torch.zeros(5,dtype=torch.long)
#         imglabel[self.trainLables.loc[self.trainLables.image==fileName ,'level'].values[0]] = 1
        
#         xtemp = self.transform(self.X[idx])
#         ytemp = self.transform(self.y[idx])
        return img ,self.trainLables[idx]

class EyeDatasetFromImages(torch.utils.data.Dataset):
    
    def __init__(self,X,y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.size(0)
    
    def __getitem__(self,idx):
        return self.X[idx],self.y[idx]

In [24]:
batch_size = 8
num_epochs = 1
num_classes = 5
feature_extract = False

In [7]:
def get_input(idx):
        imgtemp = Image.open(images_train_list[idx])
#         if self.transform!=None :
#             imgtemp = self.transform(imgtemp)
        img = transforms.ToTensor()(imgtemp)
        fileName = images_train_list[idx].split('/')[-1].split('.')[0]
        imglabel = torch.zeros(5,dtype=torch.long)
        imglabel[trainLabels.loc[trainLabels.image==fileName ,'level'].values[0]] = 1
        return img ,imglabel

In [8]:
def get_pred_for_one_image(idx,model):
        test , test_label = get_input(idx)
        test = test.view(-1,3,224,224)
        test_label = test_label.view(-1,5)
        test = test.to(device)
        test_label = test_label.to(device)
        print("test_label",test_label)
        pred = model(test)
        print("pred",pred)

In [9]:
def get_labels(filenames,trainLabels):
    labels = []
    for file in filenames:
        ft = file.split('/')[-1].split('.')[0]
        imglabel = torch.zeros(5,dtype=torch.long)
        imglabel[trainLabels.loc[trainLabels.image==ft ,'level'].values[0]] = 1
        labels.append(imglabel)
    return labels

In [10]:
f = open("train_balanced_name","r")

In [11]:
lines = f.readlines()

In [12]:
total_balanced_images = []
for line in lines:
    temp = line.strip()
    temp = './train/train_224/' + temp
    total_balanced_images.append(temp)

In [53]:
images_train_list , images_test_list = train_test_split(total_balanced_images,test_size=0.1,random_state=16)

In [54]:
dataloaders_dict = { 'train' : torch.utils.data.DataLoader(EyeDataset(images_train_list,get_labels(images_train_list,trainLabels)),batch_size=batch_size) }
dataloaders_dict['val'] = torch.utils.data.DataLoader(EyeDataset(images_test_list,get_labels(images_test_list,trainLabels)),batch_size=batch_size)

In [45]:
def train_model(model, dataloaders, criterion, optimizer, checkpoint,num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        
        
        # Each epoch has a training and validation phase
        for phase in ['train','val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            num_b = 0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                num_b += 1
                inputs = inputs.to(device)
                labels = labels.to(device)
#                 print("input size ",inputs.size())
#                 print("label size",labels.size())
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, torch.max(labels,1)[1])

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == torch.max(labels,1)[1])
                
                if num_b%35==0 :
                    print('loss in {} batch is {:.4f}'.format(num_b,loss))
                    torch.save({
                        'epoch': epoch,
                        'loss' : loss,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        }, checkpoint)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [16]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet50
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    return model_ft , input_size

In [17]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [18]:
# Initialize the model for this run
# model_ft, input_size = initialize_model("resnet", num_classes, feature_extract, use_pretrained=True)
model_ft = torch.load("resnet_balanced7pm")
# model_toy = toy_model()
# model_toy.classifier[6] = nn.Linear(4096,num_classes)
checkpoint = "resnet50.checkpoint"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

In [19]:
lr = 0.001  # 0.1
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9, nesterov=True, weight_decay=0.0001)
is_train = True
is_pretrain = True
acc_best = 0
total_epoch = 1

In [20]:
# # model = ResidualAttentionModel().cuda()
# model = torch.load('residual_an_6pm.model')
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# print(model)

In [55]:
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer,
                             "balanced_resnet.checkpoint", num_epochs=num_epochs, is_inception=False)

Epoch 0/0
----------
11907
loss in 35 batch is 0.1899
loss in 70 batch is 0.0726
loss in 105 batch is 0.2963
loss in 140 batch is 0.1011
loss in 175 batch is 0.2417
loss in 210 batch is 0.5797
loss in 245 batch is 0.0530
loss in 280 batch is 0.3242
loss in 315 batch is 0.0097
loss in 350 batch is 0.7169
loss in 385 batch is 0.2243
loss in 420 batch is 0.1239
loss in 455 batch is 0.1275
loss in 490 batch is 0.1163
loss in 525 batch is 0.0509
loss in 560 batch is 0.0227
loss in 595 batch is 0.1120
loss in 630 batch is 0.4153
loss in 665 batch is 0.2671
loss in 700 batch is 0.1432
loss in 735 batch is 0.0470
loss in 770 batch is 0.2445
loss in 805 batch is 0.0478
loss in 840 batch is 0.3638
loss in 875 batch is 0.0754
loss in 910 batch is 0.2449
loss in 945 batch is 0.4990
loss in 980 batch is 0.0745
loss in 1015 batch is 0.1048
loss in 1050 batch is 0.0429
loss in 1085 batch is 0.1169
loss in 1120 batch is 0.0283
loss in 1155 batch is 0.2421
loss in 1190 batch is 0.0328
loss in 1225 batc

In [ ]:
torch.save(model_ft,"resnet_balanced7pm")

In [56]:
torch.save(model_ft,"resnet_final12pm")